In [3]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root = './mnist_data/',
                             train = True,
                             transform = transforms.ToTensor(),
                             download = True)

test_dataset = datasets.MNIST(root = './mnist_data/',
                             train = False,
                             transform = transforms.ToTensor())

# Data Loader 
train_loader = data.DataLoader(dataset = train_dataset,
                              batch_size = batch_size,
                              shuffle = True)
test_loader = data.DataLoader(dataset = test_dataset,
                             batch_size = batch_size,
                             shuffle = False)


Training MNIST Model on cpu


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28) -> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


In [5]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # Print
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [6]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


In [7]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.311743
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.311398
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.303400
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.314534
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.316975
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.311316
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.309283
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.302307
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.301988
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.304562
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.308523
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.296128
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.299294
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.301234
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.301702
Train Epoch: 1 | Batch Status: 9600/60

Train Epoch: 2 | Batch Status: 19200/60000 (32%) | Loss: 0.668020
Train Epoch: 2 | Batch Status: 19840/60000 (33%) | Loss: 0.605097
Train Epoch: 2 | Batch Status: 20480/60000 (34%) | Loss: 0.769416
Train Epoch: 2 | Batch Status: 21120/60000 (35%) | Loss: 0.574549
Train Epoch: 2 | Batch Status: 21760/60000 (36%) | Loss: 0.525862
Train Epoch: 2 | Batch Status: 22400/60000 (37%) | Loss: 0.664618
Train Epoch: 2 | Batch Status: 23040/60000 (38%) | Loss: 0.630330
Train Epoch: 2 | Batch Status: 23680/60000 (39%) | Loss: 0.682291
Train Epoch: 2 | Batch Status: 24320/60000 (41%) | Loss: 0.932880
Train Epoch: 2 | Batch Status: 24960/60000 (42%) | Loss: 0.525883
Train Epoch: 2 | Batch Status: 25600/60000 (43%) | Loss: 0.737651
Train Epoch: 2 | Batch Status: 26240/60000 (44%) | Loss: 0.687828
Train Epoch: 2 | Batch Status: 26880/60000 (45%) | Loss: 0.464310
Train Epoch: 2 | Batch Status: 27520/60000 (46%) | Loss: 0.352714
Train Epoch: 2 | Batch Status: 28160/60000 (47%) | Loss: 0.554172
Train Epoc

Train Epoch: 3 | Batch Status: 37760/60000 (63%) | Loss: 0.398199
Train Epoch: 3 | Batch Status: 38400/60000 (64%) | Loss: 0.470699
Train Epoch: 3 | Batch Status: 39040/60000 (65%) | Loss: 0.246517
Train Epoch: 3 | Batch Status: 39680/60000 (66%) | Loss: 0.435483
Train Epoch: 3 | Batch Status: 40320/60000 (67%) | Loss: 0.436764
Train Epoch: 3 | Batch Status: 40960/60000 (68%) | Loss: 0.377630
Train Epoch: 3 | Batch Status: 41600/60000 (69%) | Loss: 0.507303
Train Epoch: 3 | Batch Status: 42240/60000 (70%) | Loss: 0.572427
Train Epoch: 3 | Batch Status: 42880/60000 (71%) | Loss: 0.351430
Train Epoch: 3 | Batch Status: 43520/60000 (72%) | Loss: 0.366165
Train Epoch: 3 | Batch Status: 44160/60000 (74%) | Loss: 0.213867
Train Epoch: 3 | Batch Status: 44800/60000 (75%) | Loss: 0.522221
Train Epoch: 3 | Batch Status: 45440/60000 (76%) | Loss: 0.416126
Train Epoch: 3 | Batch Status: 46080/60000 (77%) | Loss: 0.532777
Train Epoch: 3 | Batch Status: 46720/60000 (78%) | Loss: 0.240879
Train Epoc

Train Epoch: 4 | Batch Status: 56960/60000 (95%) | Loss: 0.241650
Train Epoch: 4 | Batch Status: 57600/60000 (96%) | Loss: 0.190648
Train Epoch: 4 | Batch Status: 58240/60000 (97%) | Loss: 0.109561
Train Epoch: 4 | Batch Status: 58880/60000 (98%) | Loss: 0.587798
Train Epoch: 4 | Batch Status: 59520/60000 (99%) | Loss: 0.242309
Training time: 0m 16s
Test set: Average loss: 0.0040, Accuracy: 9219/10000 (92%)
Testing time: 0m 17s
Train Epoch: 5 | Batch Status: 0/60000 (0%) | Loss: 0.257158
Train Epoch: 5 | Batch Status: 640/60000 (1%) | Loss: 0.242387
Train Epoch: 5 | Batch Status: 1280/60000 (2%) | Loss: 0.158387
Train Epoch: 5 | Batch Status: 1920/60000 (3%) | Loss: 0.520817
Train Epoch: 5 | Batch Status: 2560/60000 (4%) | Loss: 0.323188
Train Epoch: 5 | Batch Status: 3200/60000 (5%) | Loss: 0.358452
Train Epoch: 5 | Batch Status: 3840/60000 (6%) | Loss: 0.212551
Train Epoch: 5 | Batch Status: 4480/60000 (7%) | Loss: 0.185032
Train Epoch: 5 | Batch Status: 5120/60000 (9%) | Loss: 0.144

Train Epoch: 6 | Batch Status: 14720/60000 (25%) | Loss: 0.109803
Train Epoch: 6 | Batch Status: 15360/60000 (26%) | Loss: 0.148865
Train Epoch: 6 | Batch Status: 16000/60000 (27%) | Loss: 0.157271
Train Epoch: 6 | Batch Status: 16640/60000 (28%) | Loss: 0.240319
Train Epoch: 6 | Batch Status: 17280/60000 (29%) | Loss: 0.112769
Train Epoch: 6 | Batch Status: 17920/60000 (30%) | Loss: 0.148524
Train Epoch: 6 | Batch Status: 18560/60000 (31%) | Loss: 0.127195
Train Epoch: 6 | Batch Status: 19200/60000 (32%) | Loss: 0.258864
Train Epoch: 6 | Batch Status: 19840/60000 (33%) | Loss: 0.152398
Train Epoch: 6 | Batch Status: 20480/60000 (34%) | Loss: 0.185749
Train Epoch: 6 | Batch Status: 21120/60000 (35%) | Loss: 0.122520
Train Epoch: 6 | Batch Status: 21760/60000 (36%) | Loss: 0.131276
Train Epoch: 6 | Batch Status: 22400/60000 (37%) | Loss: 0.061157
Train Epoch: 6 | Batch Status: 23040/60000 (38%) | Loss: 0.128782
Train Epoch: 6 | Batch Status: 23680/60000 (39%) | Loss: 0.150431
Train Epoc

Train Epoch: 7 | Batch Status: 33280/60000 (55%) | Loss: 0.174727
Train Epoch: 7 | Batch Status: 33920/60000 (57%) | Loss: 0.092096
Train Epoch: 7 | Batch Status: 34560/60000 (58%) | Loss: 0.095848
Train Epoch: 7 | Batch Status: 35200/60000 (59%) | Loss: 0.095330
Train Epoch: 7 | Batch Status: 35840/60000 (60%) | Loss: 0.122317
Train Epoch: 7 | Batch Status: 36480/60000 (61%) | Loss: 0.130387
Train Epoch: 7 | Batch Status: 37120/60000 (62%) | Loss: 0.130433
Train Epoch: 7 | Batch Status: 37760/60000 (63%) | Loss: 0.190078
Train Epoch: 7 | Batch Status: 38400/60000 (64%) | Loss: 0.058627
Train Epoch: 7 | Batch Status: 39040/60000 (65%) | Loss: 0.198333
Train Epoch: 7 | Batch Status: 39680/60000 (66%) | Loss: 0.047554
Train Epoch: 7 | Batch Status: 40320/60000 (67%) | Loss: 0.090370
Train Epoch: 7 | Batch Status: 40960/60000 (68%) | Loss: 0.074229
Train Epoch: 7 | Batch Status: 41600/60000 (69%) | Loss: 0.073135
Train Epoch: 7 | Batch Status: 42240/60000 (70%) | Loss: 0.121993
Train Epoc

Train Epoch: 8 | Batch Status: 51840/60000 (86%) | Loss: 0.094631
Train Epoch: 8 | Batch Status: 52480/60000 (87%) | Loss: 0.148708
Train Epoch: 8 | Batch Status: 53120/60000 (88%) | Loss: 0.164442
Train Epoch: 8 | Batch Status: 53760/60000 (90%) | Loss: 0.044825
Train Epoch: 8 | Batch Status: 54400/60000 (91%) | Loss: 0.115247
Train Epoch: 8 | Batch Status: 55040/60000 (92%) | Loss: 0.097088
Train Epoch: 8 | Batch Status: 55680/60000 (93%) | Loss: 0.107191
Train Epoch: 8 | Batch Status: 56320/60000 (94%) | Loss: 0.033839
Train Epoch: 8 | Batch Status: 56960/60000 (95%) | Loss: 0.083385
Train Epoch: 8 | Batch Status: 57600/60000 (96%) | Loss: 0.398748
Train Epoch: 8 | Batch Status: 58240/60000 (97%) | Loss: 0.097746
Train Epoch: 8 | Batch Status: 58880/60000 (98%) | Loss: 0.061618
Train Epoch: 8 | Batch Status: 59520/60000 (99%) | Loss: 0.088802
Training time: 0m 16s
Test set: Average loss: 0.0018, Accuracy: 9652/10000 (97%)
Testing time: 0m 17s
Train Epoch: 9 | Batch Status: 0/60000 (